In [27]:
import pandas as pd
import numpy as np
from pathlib import Path

base = '../datasets/Countix/'

In [28]:
df = pd.read_csv(open(Path(base, 'countix_val.csv')))
df.groupby('class').count().sort_values(by='count', ascending=False)

,video_id,kinetics_start,kinetics_end,repetition_start,repetition_end,count
class,,,,,,
doing aerobics,62,62,62,62,62,62
bouncing ball (not juggling),53,53,53,53,53,53
slicing onion,51,51,51,51,51,51
battle rope training,49,49,49,49,49,49
rope pushdown,47,47,47,47,47,47
mountain climber (exercise),46,46,46,46,46,46
triple jump,45,45,45,45,45,45
front raises,43,43,43,43,43,43
exercising arm,42,42,42,42,42,42


In [4]:
def df2link(df):
    return 'https://youtube.com/watch?v=' + df['video_id'] + '&t=' + \
        df['kinetics_start'].astype(str) + 's'

In [5]:
from IPython.display import HTML

cate = df[df['class'] == 'situp'].sample(n=1)
x = df2link(cate).item()
HTML(f"""<a href={x}>{x}</a>""")

In [6]:
actions = ['exercising arm', 'bench pressing', 'front raises', 'squat', 'jumping jacks',
     'lunge', 'mountain climber (exercise)', 'pull ups', 'push up', 'situp']

# df[df['class'].isin(actions)].to_csv('workouts_val.csv')

## Download

In [13]:
%pip install yt-dlp -q
import yt_dlp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def download_ytb(vid, folder):
    url = 'https://www.youtube.com/watch?v='
    link = url + vid
    ydl_opts = {
        'outtmpl': f'{folder}/{vid}',
        'quiet': True,
        'ignoreerrors': True,
        'format': 136, # 136 for mp4 1280x720 30 fps no audio 
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])

# TODO use multi-thread
df.apply(lambda row: download_ytb(row['video_id'], 'downloads'), axis=1)

## Process videos

In [ ]:
import os
import subprocess

vids = os.listdir('/content/ALL')
dest = '/content/clips/'
print(len(vids))
def cut():
    for vid in vids:
        print(vid)
        v = df[df['video_id'] == vid.split('.')[0]].iloc[0]
        if not len(v): # .DS_store
            continue
        name = v['video_id']
        ss = v['kinetics_start']
        to = v['kinetics_end']
        infile = os.path.join('/content/ALL/', vid)
        outfile = os.path.join(dest, name+'.mp4')
        cmd = f"ffmpeg -i {infile} -ss {ss} -t 10 -an {outfile}"
        print(cmd)
        subprocess.run([cmd], shell=True, capture_output=True)

## Extract frames

In [29]:
df_train = pd.read_csv(Path(base, 'workouts_train.csv'))
df_val = pd.read_csv(Path(base, 'workouts_val.csv'))
print('total', len(df_train), len(df_val))
print('unique', df_train['video_id'].nunique(), df_val['video_id'].nunique())

# intersection of df_train and df_val video_id
len(df_train[df_train['video_id'].isin(df_val['video_id'])])

total 1092 369
unique 1041 349


0

In [33]:
# view duplicates
df_train[df_train['video_id'].duplicated()].sort_values('video_id').head()

,index,video_id,class,kinetics_start,kinetics_end,repetition_start,repetition_end,count
273,1011,0_EeIVIi5vU,front raises,12,22,18.423090,21.968302,2
1008,3376,1pBelFN3flQ,situp,1,11,7.149733,10.993316,3
902,2726,2-m13VKbJbU,push up,5,15,12.841829,14.969133,2
514,1818,2gxsQ5kpbXY,squat,6,16,11.972639,15.976643,2
908,2732,4s6xVt9PPIM,push up,71,81,77.473140,80.943277,2


In [12]:
# extract frames from videos
# images are in folder with name of video. image name format 'img_{:05d}.jpg'
import subprocess
import os.path as osp

def extract_frames(video_dir, output_dir):
    for name in os.listdir(video_dir):
        folder_name = name.split('.')[0]
        dest = osp.join(output_dir,  folder_name)
        if osp.exists(dest):
            continue
        mkdir_cmd = f'mkdir "{dest}"'
        subprocess.run(mkdir_cmd, shell=True)
        cmd = f'ffmpeg -i {video_dir}/{name} {output_dir}/{folder_name}/img_%05d.jpg'
        subprocess.run(cmd, shell=True)


In [ ]:
train_dir = 'datasets/countix_train_clips'
val_dir = 'datasets/countix_val_clips'
# extract_frames(train_dir, 'data/countix/rawframes/train')
# extract_frames(val_dir, 'data/countix/rawframes/val')

### Build file list

- rawframes
  - train
    - video_id
      - img_%5d.jpg
  - val
    - video_id
      - img_%5d.jpg
- rawframes_train.txt [video_id frames classid]
- rawframes_val.txt [video_id frames classid]

In [34]:
actions = ['exercising arm', 'bench pressing', 'front raises', 'squat', 'jumping jacks',
     'lunge', 'mountain climber (exercise)', 'pull ups', 'push up', 'situp']
df_train = pd.read_csv(Path(base, 'workouts_train.csv'))
df_val = pd.read_csv(Path(base, 'workouts_val.csv'))

df_train.head()

,index,video_id,class,kinetics_start,kinetics_end,repetition_start,repetition_end,count
0,104,u45qCLuwa7I,exercising arm,38,48,45.941275,47.943277,9
1,130,BFPKPjBgXIU,bench pressing,22,32,22.000000,32.000000,3
2,131,DnMx7EjjmFo,bench pressing,11,21,14.160000,21.000000,5
3,132,ExiJ7qKakH4,bench pressing,0,10,0.000000,9.977940,6
4,133,E9KZOVuBRfA,bench pressing,35,45,35.000000,45.000000,11


Repetition start and end seconds to frames

In [24]:
def sec2frame(row, rep='start'):
    offset = row['kinetics_start']
    if rep == 'start':
        sec = row['repetition_start']
    elif rep == 'end':
        sec = row['repetition_end']
    return int((sec-offset) * 30)

df_train['rep_start'] = df_train.apply(lambda x: sec2frame(x, rep='start'), axis=1)
df_train['rep_end'] = df_train.apply(lambda x: sec2frame(x, rep='end'), axis=1)
df_val['rep_start'] = df_val.apply(lambda x: sec2frame(x, rep='start'), axis=1)
df_val['rep_end'] = df_val.apply(lambda x: sec2frame(x, rep='end'), axis=1)
df_train.head()

,index,video_id,class,kinetics_start,kinetics_end,repetition_start,repetition_end,count,rep_start,rep_end
0,104,u45qCLuwa7I,exercising arm,38,48,45.941275,47.943277,9,238,298
1,130,BFPKPjBgXIU,bench pressing,22,32,22.000000,32.000000,3,0,300
2,131,DnMx7EjjmFo,bench pressing,11,21,14.160000,21.000000,5,94,300
3,132,ExiJ7qKakH4,bench pressing,0,10,0.000000,9.977940,6,0,299
4,133,E9KZOVuBRfA,bench pressing,35,45,35.000000,45.000000,11,0,300


Save to new csv

In [26]:
df_train.to_csv(Path(base, 'workouts_train_rep.csv'))
df_val.to_csv(Path(base, 'workouts_val_rep.csv'))

In [70]:
def get_label(vid, df):
    return df[df['video_id'] == vid]['class'].unique()[0]

def build_file_list(rawframes_dir, input_txt, output_txt):
    for name in os.listdir(rawframes_dir):
        num_frames = len(os.listdir(osp.join(rawframes_dir, name)))
        d = get_label(input_txt)
        label = d[name]
        with open(output_txt, 'a') as f:
            f.write(f'{name} {num_frames} {label}\n')

data_dir = 'data/countix/'
# save class to labels.txt
with open(osp.join(data_dir, 'labels.txt'), 'w') as f:
    for c in actions:
        f.write(f'{c}\n')

# build video file list 
train_video_txt = osp.join(data_dir,'videos', 'train.txt')
val_video_txt = osp.join(data_dir,'videos', 'val.txt')
with open(train_video_txt, 'w') as f:
    for vid in df_train['video_id'].unique():
        f.write(f'{vid} {actions.index(get_label(vid, df_train))}\n')

with open(val_video_txt, 'w') as f:
    for vid in df_val['video_id'].unique():
        f.write(f'{vid} {actions.index(get_label(vid, df_val))}\n')

In [74]:
# build rawframes train.txt [video_id, num_frames, label]
train_rawframes_txt = osp.join(data_dir, 'rawframes', 'train.txt')
with open(train_rawframes_txt, 'w') as f:
    files = os.listdir(osp.join(data_dir, 'rawframes', 'train'))
    for v in files:
        num_frames = len(os.listdir(osp.join(data_dir, 'rawframes', 'train', v)))
        label = actions.index(get_label(v, df_train))
        f.write(f'{v} {num_frames} {label}\n')

# build rawframes val.txt [video_id, num_frames, label]
val_rawframes_txt = osp.join(data_dir, 'rawframes', 'val.txt')
with open(val_rawframes_txt, 'w') as f:
    files = os.listdir(osp.join(data_dir, 'rawframes', 'val'))
    for v in files:
        num_frames = len(os.listdir(osp.join(data_dir, 'rawframes', 'val', v)))
        label = actions.index(get_label(v, df_val))
        f.write(f'{v} {num_frames} {label}\n')